# **Point Sampling (CPS and FPS)**

In this notebook, the following steps are performed:

- Load each CAD model as a point cloud  
- Select 50 keypoints using Curvature Point Sampling (CPS)  
- Select another 50 keypoints using Farthest Point Sampling (FPS)  
- Save both sets of keypoints to a JSON file  
- Visualize the selected keypoints side-by-side on the model for quick comparison  


## **Importing required libraries and modules**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install open3d
!pip install -U kaleido

In [3]:
import json
import os
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm

## **CPS (Curvature Point Sampling)**

The script scans a folder of `.ply` models, turns each one into a cloud of 3D points, and measures how curved or geometrically significant every point is by looking at its nearby neighbors.

It then selects K keypoints per model that are both highly distinctive (based on curvature) and evenly distributed across the surface, ensuring a small but informative set of important surface points.

Finally, it saves these keypoints for each object into one JSON file.

In [4]:
def estimate_curvature(points, k=30):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=k))

    # Build KDTree for neighborhood search
    kdtree = o3d.geometry.KDTreeFlann(pcd)

    curvatures = []
    for i in range(len(points)):
        _, idx, _ = kdtree.search_knn_vector_3d(pcd.points[i], k)
        if len(idx) < 3:
            curvatures.append(0)
            continue
        neighbors = np.asarray(pcd.points)[idx, :]
        covariance = np.cov(neighbors.T)
        eigenvalues = np.linalg.eigvalsh(covariance)
        curvature = eigenvalues[0] / (sum(eigenvalues) + 1e-8)  # Normalize
        curvatures.append(curvature)

    return np.array(curvatures)

def cps_sampling(points, curvatures, K=50):
    # Normalize curvature
    curvatures = np.array(curvatures)
    curvatures = (curvatures - curvatures.min()) / (curvatures.max() - curvatures.min() + 1e-8)

    # Histogram equalization
    hist, bins = np.histogram(curvatures, bins=256, density=True)
    cdf = hist.cumsum()
    cdf = (cdf - cdf.min()) / (cdf.max() - cdf.min() + 1e-8)
    equalized = np.interp(curvatures, bins[:-1], cdf)

    # Score initialization
    selected_indices = []
    weights = equalized
    centroid = np.mean(points, axis=0)
    d_home = np.linalg.norm(points - centroid, axis=1)
    score = d_home * weights

    for k in range(K):
        score[selected_indices] = -np.inf  # suppress already picked
        idx = np.argmax(score)
        selected_indices.append(idx)
        ref_point = points[idx]
        d_temp = np.linalg.norm(points - ref_point, axis=1)
        score = np.minimum(score, d_temp * weights)

    return points[selected_indices]

def extract_cps_keypoints(cad_model_dir, output_json_path, skip_classes=None, K=50):
    keypoints_per_class = {}
    all_models = sorted(os.listdir(cad_model_dir))

    for model_file in tqdm(all_models):
        if model_file.endswith(".ply"):
            class_name = model_file.replace("obj_", "").replace(".ply", "")
            if skip_classes and model_file in skip_classes:
                continue

            mesh_path = os.path.join(cad_model_dir, model_file)
            pcd = o3d.io.read_point_cloud(mesh_path)
            points = np.asarray(pcd.points)

            curvatures = estimate_curvature(points)
            keypoints = cps_sampling(points, curvatures, K=K)

            keypoints_per_class[class_name] = keypoints.tolist()

    with open(output_json_path, "w") as f:
        json.dump(keypoints_per_class, f, indent=4)
    print(f"\nSaved CPS keypoints to: {output_json_path}")

In [ ]:
cad_model_dir = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/raw/models"
output_json = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_cps.json"
skip = ["obj_03.ply", "obj_07.ply", "models_info.yml"]

extract_cps_keypoints(cad_model_dir, output_json_path=output_json, skip_classes=skip)


100%|██████████| 16/16 [00:36<00:00,  2.26s/it]
Saved CPS keypoints to: /content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_cps.json



## **3D Visualization of CPS Keypoints**

In [7]:
def visualize_keypoints(obj_class_name, keypoints_json_path, cad_model_dir, method_name="Keypoints"):
    # Load keypoints
    with open(keypoints_json_path, "r") as f:
        keypoints_data = json.load(f)

    if obj_class_name not in keypoints_data:
        raise ValueError(f"Class '{obj_class_name}' not found in keypoints JSON.")

    keypoints = np.array(keypoints_data[obj_class_name])

    # Load model
    model_file = f"obj_{obj_class_name}.ply"
    mesh_path = os.path.join(cad_model_dir, model_file)
    if not os.path.exists(mesh_path):
        raise FileNotFoundError(f"Model file '{model_file}' not found in '{cad_model_dir}'.")

    mesh = o3d.io.read_point_cloud(mesh_path)
    points = np.asarray(mesh.points)

    # Plot
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode='markers',
        marker=dict(size=2, color='blue'),
        name='All Points'
    ))

    fig.add_trace(go.Scatter3d(
        x=keypoints[:, 0], y=keypoints[:, 1], z=keypoints[:, 2],
        mode='markers',
        marker=dict(size=6, color='red'),
        name=f'{method_name} Keypoints'
    ))

    fig.update_layout(
        title=f"{method_name} Keypoints for Class '{obj_class_name}' ({model_file})",
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
        width=800, height=600
    )

    fig.show()


In [ ]:
cad_model_dir = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/raw/models"
keypoints_json = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_cps.json"

# Visualize object index 0 (corresponds to 'obj_01.ply')
visualize_keypoints('01', keypoints_json, cad_model_dir, "CPS")

## **FPS (Farthest Point Sampling)**

In this step, Farthest Point Sampling (FPS) is performed on each 3D model to select K keypoints that are evenly distributed across the object's surface. The selected keypoints are stored by class name and saved to a single JSON file.

In [ ]:
def fps_sampling(points, K):

    sampled = [np.random.randint(len(points))]  # Start with a random point
    distances = np.full(len(points), np.inf)

    for _ in range(1, K):
        last_point = points[sampled[-1]]
        dist = np.linalg.norm(points - last_point, axis=1)
        distances = np.minimum(distances, dist)
        next_idx = np.argmax(distances)
        sampled.append(next_idx)

    return points[sampled]

def extract_fps_keypoints(cad_model_dir, output_json_path, skip_classes=None, K=50):
    keypoints_per_class = {}
    all_models = sorted(os.listdir(cad_model_dir))

    for model_file in tqdm(all_models):
        if model_file.endswith(".ply"):
            if skip_classes and model_file in skip_classes:
                continue

            class_name = model_file.replace("obj_", "").replace(".ply", "")
            model_path = os.path.join(cad_model_dir, model_file)
            pcd = o3d.io.read_point_cloud(model_path)
            points = np.asarray(pcd.points)

            fps_points = fps_sampling(points, K)
            keypoints_per_class[class_name] = fps_points.tolist()

    with open(output_json_path, "w") as f:
        json.dump(keypoints_per_class, f, indent=4)
    print(f"\nSaved FPS keypoints to: {output_json_path}")

In [ ]:
cad_model_dir = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/raw/models"
output_path = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_fps.json"
skip = ["obj_03.ply", "obj_07.ply", "models_info.yml"]

extract_fps_keypoints(cad_model_dir, output_path, skip_classes=skip)


100%|██████████| 16/16 [00:00<00:00, 19.81it/s]
Saved FPS keypoints to: /content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_fps.json



## **3D Visualization of FPS Keypoints**

In [ ]:
cad_model_dir = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/raw/models"
keypoints_json = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_fps.json"

# Visualize object index 0 (corresponds to 'obj_01.ply')
visualize_keypoints('01', keypoints_json, cad_model_dir, "FPS")

## **Comparison Visualization (FPS Vs CPS)**

In [10]:
def visualize_cps_vs_fps(class_name, cps_json_path, fps_json_path, cad_model_dir):

    # Load CPS and FPS keypoints
    with open(cps_json_path, "r") as f:
        cps_data = json.load(f)
    with open(fps_json_path, "r") as f:
        fps_data = json.load(f)

    if class_name not in cps_data or class_name not in fps_data:
        raise ValueError(f"Class '{class_name}' not found in one or both keypoint files.")

    cps_points = np.array(cps_data[class_name])
    fps_points = np.array(fps_data[class_name])

    # Load original point cloud
    model_file = f"obj_{class_name}.ply"
    mesh_path = os.path.join(cad_model_dir, model_file)
    if not os.path.exists(mesh_path):
        raise FileNotFoundError(f"Model file '{model_file}' not found in directory '{cad_model_dir}'.")

    mesh = o3d.io.read_point_cloud(mesh_path)
    points = np.asarray(mesh.points)

    # Plot everything
    fig = go.Figure()

    # All points
    fig.add_trace(go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode='markers',
        marker=dict(size=2, color='blue'),
        name='All Points'
    ))

    # CPS points (red)
    fig.add_trace(go.Scatter3d(
        x=cps_points[:, 0], y=cps_points[:, 1], z=cps_points[:, 2],
        mode='markers',
        marker=dict(size=6, color='red'),
        name='CPS Keypoints'
    ))

    # FPS points (green)
    fig.add_trace(go.Scatter3d(
        x=fps_points[:, 0], y=fps_points[:, 1], z=fps_points[:, 2],
        mode='markers',
        marker=dict(size=6, color='green'),
        name='FPS Keypoints'
    ))

    fig.update_layout(
        title=f"Class '{class_name}' — CPS (Red) vs FPS (Green) Keypoints",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        width=900,
        height=700
    )

    fig.show()

In [11]:
cad_model_dir = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/raw/models"
cps_json = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_cps.json"
fps_json = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation/data/point_sampling_data/3D_50_keypoints_fps.json"

# Visualize class 0 (obj_01)
visualize_cps_vs_fps('01', cps_json, fps_json, cad_model_dir)